#**Currency Classification Deep Learning Model Using Convolutional Neural Networks**

*Necessary Libraries*

In [ ]:
!pip install tensorflow

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import numpy as np


*Access Google Drive for Dataset*

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Define base directory
base_dir = "/content/drive/MyDrive/DL/Currency/dataset/old-currency"

# Define train, test, and valid directories
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
valid_dir = os.path.join(base_dir, "valid")

In [4]:
img_width, img_height = 200, 200
batch_size = 32
epochs = 10
num_classes = 4  # Number of classes (number of different currency notes)



In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Ensure that images are not shuffled
)


Found 635 images belonging to 4 classes.
Found 118 images belonging to 4 classes.
Found 233 images belonging to 4 classes.


In [6]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
import matplotlib.pyplot as plt

# Display images from the test generator
class_labels = list(train_generator.class_indices.keys())
for images, labels in test_generator:
    # Display each image in the batch
    for i in range(len(images)):
        plt.imshow(images[i])
        plt.title(f'Class: {class_labels[np.argmax(labels[i])]}')
        plt.axis('off')
        plt.show()

In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/10
20/20 [==============================] - 311s 15s/step - loss: 1.4032 - accuracy: 0.2551 - val_loss: 1.4823 - val_accuracy: 0.2034
Epoch 2/10
20/20 [==============================] - 88s 4s/step - loss: 1.3552 - accuracy: 0.3449 - val_loss: 1.3482 - val_accuracy: 0.3644
Epoch 3/10
20/20 [==============================] - 80s 4s/step - loss: 1.1003 - accuracy: 0.5480 - val_loss: 1.1146 - val_accuracy: 0.4831
Epoch 4/10
20/20 [==============================] - 74s 4s/step - loss: 0.8906 - accuracy: 0.6362 - val_loss: 1.2192 - val_accuracy: 0.6102
Epoch 5/10
20/20 [==============================] - 81s 4s/step - loss: 0.8332 - accuracy: 0.6598 - val_loss: 1.2756 - val_accuracy: 0.5847
Epoch 6/10
20/20 [==============================] - 75s 4s/step - loss: 0.6793 - accuracy: 0.7276 - val_loss: 1.2015 - val_accuracy: 0.6017
Epoch 7/10
20/20 [==============================] - 75s 4s/step - loss: 0.5052 - accuracy: 0.8016 - val_loss: 1.7616 - val_accuracy: 0.5678
Epoch 8/10
20/20 [

In [11]:

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print()
print(f'Test accuracy: {test_accuracy*100}')
print(f'Test loss: {test_loss}')


8/8 [==============================] - 66s 9s/step - loss: 1.0852 - accuracy: 0.6867

Test accuracy: 68.66952776908875
Test loss: 1.0852299928665161


In [ ]:
# Get the number of batches in the test generator
num_batches = len(test_generator)

# Generate random indices of batches to sample
random_indices = random.sample(range(num_batches), min(num_images_to_test, num_batches))

# Iterate through the randomly selected batches
for i in random_indices:
    # Get the i-th batch of images and labels
    images, labels = test_generator[i]

    # Generate predictions for the batch
    predictions = model.predict(images)

    # Display predictions along with images
    for j in range(len(images)):
        plt.figure()
        plt.imshow(images[j])  # Display the j-th image in the batch
        predicted_class = class_labels[np.argmax(predictions[j])]
        plt.title(f'Image {i*num_images_to_test + j + 1}: Predicted class - {predicted_class}')
        plt.axis('off')
        plt.show()
